In [8]:
import os
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
from PIL import Image
import requests
import datetime
import re
import time
import urllib
from urllib.request import Request, urlopen
import pandas as pd
import numpy as np

url = "https://www.larvalabs.com/cryptopunks/bids"
#hdr = {'User-Agent': 'Mozilla/5.0'}

In [9]:
web_r = requests.get(url)
websoup = BeautifulSoup(web_r.text, "html.parser")

#req = Request(url,headers=hdr)
#web_r = urlopen(req)
#selsoup = BeautifulSoup(web_r, "html.parser")

In [10]:
final_df = pd.DataFrame()
for titlesoup in websoup.findAll("div",{"class":"col-md-1 col-sm-2 col-xs-3 punk-image-container-dense"}):
    for element in titlesoup.findAll("a"):
        final_df= final_df.append([element.get("title")])


In [11]:
final_df = pd.DataFrame(final_df[0].str.split(expand=True))

In [12]:
final_df.drop(0, axis = 1, inplace=True)
final_df[1] = final_df[1].str.replace("#", "")

In [13]:
final_df.rename({1:"punk_num"}, axis =1, inplace=True)

In [14]:
final_df

,punk_num,2,3,4,5,6,7,8,9,10,11,12,13
0,1886,For,Sale:,515,ETH,"($1,083,601.24",USD),Bid:,340,ETH,"($715,387.23",USD),None
0,1710,Bid:,88.88,ETH,"($187,010.64",USD),None,None,None,None,None,None,None
0,7488,Bid:,48.5,ETH,"($102,047.88",USD),None,None,None,None,None,None,None
0,1222,For,Sale:,69,ETH,"($145,181.53",USD),Bid:,47,ETH,"($98,891.76",USD),None
0,3645,For,Sale:,50,ETH,"($105,204.00",USD),Bid:,44,ETH,"($92,579.52",USD),None
...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,6264,Bid:,0,ETH,(<,$0.01,USD),None,None,None,None,None,None
0,3955,Bid:,0,ETH,(<,$0.01,USD),None,None,None,None,None,None
0,6347,Bid:,0,ETH,(<,$0.01,USD),None,None,None,None,None,None
0,3967,Bid:,0,ETH,(<,$0.01,USD),None,None,None,None,None,None


Price history

In [15]:
import os
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
from PIL import Image
import requests
import datetime
import re
import time
import urllib
from urllib.request import Request, urlopen
import pandas as pd
import numpy as np

url = "https://www.larvalabs.com/cryptopunks/details/1119"

In [16]:
web_r = requests.get(url)
websoup = BeautifulSoup(web_r.text, "html.parser")

In [17]:
price_info = pd.DataFrame()
for titlesoup in websoup.findAll("div",{"class":"table-responsive"}):
    for element in titlesoup.findAll("tr", {"class":re.compile(r"\bpunk-history-row\b")}):
        price_info= price_info.append([element.get_text().replace("\n", "")])

In [18]:
price_info = price_info[0].str.split(expand=True)

In [19]:
price_info["Price"] = price_info[0].str.extract('(\d+.\d+)')

In [20]:
price_info.Price = np.where(price_info.Price.str.contains(re.compile(r"[a-zA-Z]+")), 0, price_info.Price)

In [21]:
price_info.Price = price_info.Price.astype(float)

In [22]:
price_info["Price"] = np.where(price_info[0].str.contains("K"), price_info.Price*1000, price_info["Price"])
price_info["Bid"] = np.where(price_info[0].str.contains("Bid"), 1, 0)
price_info["Offered"] = np.where(price_info[0].str.contains("Offered"), 1, 0)

In [80]:
price_info["Transfer"] = np.where(price_info[0].str.contains("Transfer"), 1, 0)
price_info["Claimed"] = np.where(price_info[0].str.contains("Claimed"), 1, 0)

In [81]:
price_info["price_in_dollars"] = price_info[1].apply(lambda x: re.findall(r"\(([A-Za-z0-9_$,.<]+)\)", x))

In [82]:
price_info["Withdrawn"] = np.where(price_info[1].str.contains("Withdrawn"), 1, 0)

In [86]:
price_info["price_in_dollars"] = np.where(price_info.Withdrawn == 1, price_info[2].apply(lambda x: re.findall(r"\(([A-Za-z0-9_$,.<]+)\)", x)), price_info["price_in_dollars"])

In [88]:
price_info = price_info.explode("price_in_dollars")

In [91]:
price_info["Offer"] = np.where(price_info[0].str.contains("Offer"), 1, 0)

Year

In [98]:
price_info["Year"] = price_info[4].astype(str).apply(lambda x: re.findall(r"^[12][0-9]{3}$", x))
price_info = price_info.explode("Year")

In [103]:
price_info["Year"] = np.where(price_info.Year.isnull(), price_info[3].astype(str).apply(lambda x: re.findall(r"^[12][0-9]{3}$", x)), price_info["Year"])
price_info = price_info.explode("Year")

In [109]:
price_info["Year"] = np.where(price_info.Year.isnull(), price_info[2].astype(str).apply(lambda x: re.findall(r"^[12][0-9]{3}$", x)), price_info["Year"])
price_info = price_info.explode("Year")

Month

In [122]:
price_info["Month"] = price_info[1].astype(str).apply(lambda x: re.findall(r"\b[a-zA-Z]{3}\b", x))
price_info = price_info.explode("Month")

In [123]:
price_info["Month"] = np.where(price_info.Month.isnull(), price_info[2].astype(str).apply(lambda x: re.findall(r"\b[a-zA-Z]{3}\b", x)), price_info["Month"])
price_info = price_info.explode("Month")

In [118]:
#price_info["Month"] = np.where(price_info.Month.isnull(), price_info[1].astype(str).apply(lambda x: re.findall(r"(.{3})\s*$", x)), price_info["Month"])
#price_info = price_info.explode("Month")

In [119]:
#price_info["Month"] = np.where(price_info.Month.isnull(), price_info[0].astype(str).apply(lambda x: re.findall(r"(.{3})\s*$", x)), price_info["Month"])
#price_info = price_info.explode("Month")

In [129]:
price_info = price_info.iloc[:,6:]